In [2]:
import json
import numpy as np
import random
from tqdm import tqdm
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.decomposition import PCA
import re
import matplotlib.pyplot as plt
import sklearn

In [3]:
def rand_emot():
    e = ["(o_o)",":-)",":P",":D","x)","ᓚᘏᗢ","╯°□°）╯︵ ┻━┻",":)",
         "*<:-)","^_^","(⌐■_■)","¯\_(ツ)_/¯", "(T_T)",":o","OwO",
        "( ͡❛ ͜ʖ ͡❛)","(̶◉͛‿◉̶)","( ≖.≖)","(ㆆ_ㆆ)","ʕ•́ᴥ•̀ʔっ","( ◡́.◡̀)","(^◡^ )"]
    return random.choice(e)

def load_files():
    text_pairs = [] #Would be nice to have as np.array
    labels = []
    fandom = []
    
    pair_id = []
    true_id = []
    
    #Load truth JSON
    for line in open('data/modified/train_truth.jsonl'):
        d = json.loads(line.strip())
        labels.append(int(d['same']))
        true_id.append(d['id'])

    #Load actual fanfic.
    print("loading fanfic...",rand_emot())
    for line in tqdm(open('data/modified/train_pair.jsonl')):
        d = json.loads(line.strip())
        text_pairs.append(d['pair'])
        fandom.append(d['fandoms'])
        pair_id.append(d['id'])

    print("done loading",rand_emot())
    
    return text_pairs, labels, fandom, pair_id, true_id

In [4]:
text_pairs, labels, fandom, pair_id, true_id = load_files()

588it [00:00, 2898.09it/s]

loading fanfic... ( ≖.≖)


1578it [00:00, 2886.95it/s]

done loading :P


# Feature extraction

Word frequency and word frequency distribution

In [5]:
def frequency_distribution(text_pair): #expect untokenized input
    
    pair = []
    
    for text in text_pair: 
        tokens = nltk.word_tokenize(text) #tokenize
        
        freq_dist = nltk.FreqDist(tokens) #compute frequency distribution
        pair.append(freq_dist)
        
    return pair #return frequency distribution of each fanfic in the input pair

In [6]:
def word_freq(text_pair): #expects tokenized pairs
    fdist0 = nltk.FreqDist(text_pair[0])
    fdist1 = nltk.FreqDist(text_pair[1])
    
    return [fdist0, fdist1]

def word_freq_single(text):
    fdist = nltk.FreqDist(text)
    return fdist

def tokenize(text_pair):
    return [nltk.word_tokenize(text_pair[0]),nltk.word_tokenize(text_pair[1])]

def vector_freq_dist(freq_dists): #I don't think this works...
    return [list(freq_dists[0].values()), list(freq_dists[1].values())]

def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)+0.0000000001)

In [7]:
def create_corpus(text_pairs):
    '''input all text pairs to create a corpus'''
    corpus = [x[i] for x in text_pairs for i in range(len(x))]
    return corpus

def fit_tfidf(corpus):
    vectorizer = TfidfVectorizer()
    print("training vectorizer...",rand_emot())
    X = vectorizer.fit_transform(corpus)
    print("vectorizer fit!", rand_emot())
    
    
    df = pd.DataFrame(X[0].T.todense(), index=vectorizer.get_feature_names(), columns=["TF-IDF"])
    df = df.sort_values('TF-IDF', ascending=False)
    
    return X, df

In [8]:
corpus = create_corpus(text_pairs)

In [9]:
#tf-idf on the raw text. Likely not useful, as you can see, it is sesnitive to the fandom.
raw_tfidf, tfidf_df = fit_tfidf(corpus)

training vectorizer... (ㆆ_ㆆ)
vectorizer fit! (T_T)


In [10]:
raw_tfidf[:10]

<10x86525 sparse matrix of type '<class 'numpy.float64'>'
	with 10515 stored elements in Compressed Sparse Row format>

In [11]:
tfidf_df.head(10)

,TF-IDF
the,0.389299
to,0.295670
kuroko,0.283424
judgement,0.247891
was,0.184794
and,0.182388
it,0.182330
that,0.172474
she,0.169618
her,0.146488


In [12]:
#Attempting to perform tf-idf on only symbols.
def isolate_symbols(corpus):
    #Add \d to omit digits too.
    sym_corpus = []
    for text in corpus:
        sym_corpus.append(' '.join(re.findall("[^a-zA-Z\s]+", text)))
    return sym_corpus

symbols = isolate_symbols(corpus)

#Okay, tf-idf doesn't work with symbols. I'll convert them to made-up words

In [13]:
punct_matrix, punct_DF = fit_tfidf(symbols)

training vectorizer... (^◡^ )
vectorizer fit! *<:-)


In [14]:
punct_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

POS-tagging and Ngrams

In [15]:
## POS Tagging and ngrams
tokens = nltk.word_tokenize(corpus[0])
pos_tags = nltk.pos_tag(tokens)
pos_bigrams = nltk.bigrams(pos_tags)

LIX calculation - LIX = readability index, a measure for the readability of a text

In [16]:
def compute_lix(text):
    tokens = nltk.word_tokenize(text)
    splt = text.split()
    o = len(splt)+1
    p = len([x for x in tokens if x=='.'])+1
    l = len([x for x in tokens if len(x)>6])+1
    
    return (o/p)+((l*100)/o)

In [17]:
# for text in corpus[:10]:
#     print(compute_lix(text))

In [18]:
lix_feature = []

# for i in tqdm(range(len(text_pairs))):
#     lix = compute_lix(corpus[2*1]) - compute_lix(corpus[2*i+1])
#     lix_feature.append(np.abs(lix))
    
# lix_feature = np.vstack(lix_feature)

In [19]:
lix_feature

[]

Sentence and word length - compute sentence and word length distribution

In [20]:
def remove_symbols(text):
    sentences = re.split('[\.+|!|?]', text)
    sentences = [re.sub(r"[^\w]+", ' ', x) for x in sentences if len(x.strip()) != 0]
    return ' '.join(sentences)

def get_sent_word_length(text):
    #Function, which removes symbols and count words in sentence
    #Output: length of each sentence & length of each word
    sentences = re.split('[\.+|!|?]', text)
    sentences = [re.sub(r"[^\w]+", ' ', x) for x in sentences if len(x.strip()) != 0]
    word_sentences = [nltk.word_tokenize(x) for x in sentences]
    sentence_lengths = np.array([len(x) for x in word_sentences])
    word_lengths = np.array([len(s) for x in word_sentences for s in x])
    return sentence_lengths, word_lengths

# get_sent_word_length(corpus[0])

In [21]:
avg_sent_len = []
avg_word_len = []

# for i in tqdm(range(len(text_pairs))):
#     sent_length1, word_lengths1 = get_sent_word_length(corpus[i*2])
#     sent_length2, word_lengths2 = get_sent_word_length(corpus[i*2+1])
    
#     avg_sent = np.average(sent_length1) - np.average(sent_length2)
#     avg_word = np.average(word_lengths1) - np.average(word_lengths2)
    
#     avg_sent_len.append(np.abs(avg_sent))
#     avg_word_len.append(np.abs(avg_word))

In [22]:
# avg_sent_len = np.vstack(avg_sent_len)
# avg_word_len = np.vstack(avg_word_len)

In [23]:
# avg_sent_word_feat = np.hstack((avg_sent_len, avg_word_len))

Isolating function words

In [24]:
data = 'data/modified/train_pair.jsonl'

with open('data/function_words_clean.txt', "r") as fw:
    func_words = fw.read().split()

In [25]:
# def isolate_fw(data, f_words): #data must be json file - input must be path to file 
#     fw_in_data = []

#     for line in tqdm(open(data)):
#         function_words = []
#         d = json.loads(line.strip()) #load the json file
#         text = d.get("pair") #get the actual fanfic
#         words = text[0].split() #split fanfic into words in list
#         for word in words: 
#             if word in f_words: #if the word is a function word
#                 function_words.append(word)
                
#         stringed_function_words = " ".join(function_words)
        
#         #append all function words as one long string in a list
#         fw_in_data.append([stringed_function_words]) #fw_in_data is a list with lists
#         #each list contains a string of all function words for each pair
#         #should it be a string for each pair?
        
#     return fw_in_data


In [26]:
# test = isolate_fw(data, func_words) #why is this so fast?


In [27]:
#Trying to simplify isolation of function words - make it take text_pairs as input so we only load the files once
# and use the same kind of input in our various functions

def isolate_fw_2(data, f_words): #data must be the text_pairs from load_files()
    fw_in_data = []
    
    fw_text_pairs = []
    for pair in tqdm(data):
        fw_text_pairs = []
        for text in pair: 
            function_words = []
            
            words = text.split() #split fanfic into words in list
            
            for word in words: 
                if word in f_words: #if the word is a function word
                    function_words.append(word)
                
            stringed_function_words = " ".join(function_words) #for each fanfic in a pair, makes FW a long string. 
            fw_text_pairs.append(stringed_function_words) 
            
        #append text pairs with only their function words
        fw_in_data.append(fw_text_pairs) 
        
    return fw_in_data


In [28]:
# fw_inData_2 = isolate_fw_2(text_pairs, func_words)

In [29]:
# fw_corpus = create_corpus(fw_inData_2)

In [30]:
# FW_matrix, fw_dateframe = fit_tfidf(fw_corpus)

In [31]:
# fw_matrix = FW_matrix.toarray()

In [32]:
fw_feature = []

# for i in range(len(text_pairs)):
#     cos_sim = cosine_sim(fw_matrix[2*i], fw_matrix[2*i+1])
#     fw_feature.append(cos_sim)
# fw_feature = np.vstack(fw_feature)

Isolating profanity

In [33]:
data = 'data/modified/train_pair.jsonl'
with open('data/profanity_words_clean.txt', "r") as pr:
    prof_words = pr.read().split()
del prof_words[:4]
del prof_words[2]

In [34]:
data = 'data/modified/train_pair.jsonl'

In [35]:
def isolate_profanity(data, prof_words):
    profanity = []
    
    for pair in tqdm(data):
        profanity_pairs = []
        for text in pair:
            resultwords = []

            #d = json.loads(line.strip())
            #text = d.get("pair") 
            words = text.split() 

            resultwords  = [word for word in words if word.lower() in prof_words]

            result = " ".join(resultwords)
            profanity_pairs.append(result)
        
        profanity.append(profanity_pairs) 

        
    return profanity

In [36]:
# profanity_inData = isolate_profanity(text_pairs, prof_words)

In [37]:
# profanity_corpus = create_corpus(profanity_inData)

In [38]:
# profanity_matrix, profanity_dataframe = fit_tfidf(profanity_corpus)

In [39]:
# profanity_matrix = profanity_matrix.toarray()

In [40]:
profanity_feature = []

# for i in range(len(text_pairs)):
#     cos_sim = cosine_sim(profanity_matrix[2*i], profanity_matrix[2*i+1])
#     profanity_feature.append(cos_sim)
# profanity_feature = np.vstack(profanity_feature)

In [41]:
# features = np.hstack((fw_feature, profanity_feature))

Yule's K computations - different implementations

(a) Our own implementation - delete

In [42]:
def tokenize_no_symbols(text):
    return nltk.word_tokenize(re.sub(r'[^\w]', ' ', text))

def get_fdist_yule(text):
    text = tokenize_no_symbols(text)
    fdist = word_freq_single(text)
    return fdist
        
def get_num_unique_words(text):
    text = tokenize_no_symbols(text.lower())
    return len(set(text))

(b) Implementation below from:
https://gist.github.com/magnusnissel/d9521cb78b9ae0b2c7d6

In [43]:
import collections
import re

def tokenize(s):
    tokens = re.split(r"[^0-9A-Za-z\-'_]+", s)
    return tokens

def get_yules(s):
    """ 
    Returns a tuple with Yule's K and Yule's I.
    (cf. Oakes, M.P. 1998. Statistics for Corpus Linguistics.
    International Journal of Applied Linguistics, Vol 10 Issue 2)
    In production this needs exception handling.
    """
    tokens = tokenize(s)
    token_counter = collections.Counter(tok.upper() for tok in tokens)
    m1 = sum(token_counter.values())
    m2 = sum([freq ** 2 for freq in token_counter.values()])
    i = (m1*m1) / (m2-m1)
    k = 1/i * 10000
    return (k, i)

In [44]:
# for text in corpus[:10]: 
#     k_i = get_yules(text)
#     print(k_i)

(c) Implementation below from: https://swizec.com/blog/measuring-vocabulary-richness-with-python/

In [45]:
from nltk.stem.porter import PorterStemmer
from itertools import groupby

def words(entry):
    return filter(lambda w: len(w) > 0,
                  [w.strip("0123456789!:,.?(){}[]") for w in entry.split()])

def yule(entry):
    # yule's I measure (the inverse of yule's K measure)
    # higher number is higher diversity - richer vocabulary
    d = {}
    stemmer = PorterStemmer()
    for w in words(entry):
        w = stemmer.stem(w).lower()
        try:
            d[w] += 1
        except KeyError:
            d[w] = 1

    M1 = float(len(d))
    M2 = sum([len(list(g))*(freq**2) for freq,g in groupby(sorted(d.values()))])

    try:
        return (M1*M1)/(M2-M1)
    except ZeroDivisionError:
        return 0

In [46]:
# for text in corpus[:10]: 
#     yules_i = yule(text) #yules_i = inverse of yules K  - why do you want the inverse instead? 
#     print(yules_i)

In [47]:
yules_i_feature = []

# for i in tqdm(range(len(text_pairs))):
#     yules_i = yule(corpus[2*1]) - yule(corpus[2*i+1])
#     yules_i_feature.append(np.abs(yules_i))
    
# yules_i_feature = np.vstack(yules_i_feature)

Misspellings

In [48]:
from spellchecker import SpellChecker

def misspelled_words(text):
    #Library for spell checking
    spell = SpellChecker()
    text = remove_symbols(text)
    #Regex for finding digits
    _digits = re.compile('\d')

    #List of misspelled words
    misspelled = spell.unknown(text.split())
    #Remove words, that start with capital letter (Likely names)
    no_names = [x for x in misspelled if x.title() not in text]
    #Remove words that contain digits (7th)
    no_digits = [x for x in no_names if not bool(_digits.search(x))]
    
    #Find corrections for misspelled words - if word is more than a single character.
    corrections = [spell.correction(x) for x in no_digits if len(x)>1]
    #Remove corrections, if they have no correction (likely misclassified spelling mistake)
    remove_no_correction = [x for x in corrections if x not in misspelled]
    return remove_no_correction

misspelled_words(corpus[0])

['all']

In [49]:
misspellings_feature = []

# for i in tqdm(range(len(text_pairs))):
#     num_of_misspellings = len(misspelled_words(corpus[2*1])) - len(misspelled_words(corpus[2*i+1]))
#     misspellings_feature.append(np.abs(num_of_misspellings))
    
# misspellings_feature = np.vstack(misspellings_feature)

Save features

In [50]:
import pickle

def save_features(feature_dict):
    '''Save the updated feature dictionary. Takes dictionary as input and saves as binary file
    
    example: 
    >>> my_featues = {'freqdist': [1,6,3,5]}
    >>> save_features(my_features)'''
    
    with open('data/features.dat', 'wb') as file:
        pickle.dump(feature_dict, file)
    print("Features saved! :-)")

def load_features():
    '''Load feature dictionary. Returns the saved feature as a dictionary.
    Will then print all the available features.
    
    example: 
    >>> my_features = load_features()'''
    
    with open('data/features.dat', 'rb') as file:
        feats = pickle.load(file)
    print("Features available:")
    for i in feats.keys():
        print(i)
    
    return feats


# Classification

In [51]:
from sklearn import preprocessing

In [52]:
# feat_matrix = function words, profanity words, avg sentence length, avg word length, lix, yules i, number of misspellings
feat_matrix = np.load("feature_matrix.dat", allow_pickle=True) 

In [53]:
feat_matrix

array([[ 0.92239211,  0.03173604,  0.57456802, ...,  3.93799665,
        10.98592568, 16.        ],
       [ 0.75270986,  0.19542507,  2.14250999, ...,  3.4817081 ,
         2.18477903,  1.        ],
       [ 0.92438127,  0.        ,  0.68910175, ...,  6.82064284,
         0.57238363,  3.        ],
       ...,
       [ 0.92556229,  0.        ,  2.91769646, ...,  2.76283631,
         0.45653242,  1.        ],
       [ 0.88020293,  0.        ,  3.09249311, ...,  6.5335426 ,
         2.48188354,  3.        ],
       [ 0.7391699 ,  0.        ,  1.54155921, ...,  2.29202822,
         0.79493386,  1.        ]])

In [54]:
normalized_scalars = preprocessing.normalize(feat_matrix[:,2:])

In [55]:
normalized_scalars

array([[0.02900049, 0.00148426, 0.1987647 , 0.55449876, 0.80757693],
       [0.45075178, 0.0687992 , 0.73249886, 0.45964455, 0.21038491],
       [0.09176661, 0.03414141, 0.90829441, 0.07622344, 0.39950534],
       ...,
       [0.70018754, 0.02330171, 0.66302427, 0.10955846, 0.23997957],
       [0.37659038, 0.01869928, 0.79562644, 0.302233  , 0.36532697],
       [0.50464247, 0.08649417, 0.75031486, 0.26022834, 0.32735847]])

In [56]:
features = np.hstack((feat_matrix[:,:2],normalized_scalars))

In [57]:
without_lix = feat_matrix[:,[0,1,2,3,5]]
without_avg_word = feat_matrix[:,[0,1,2,4,5]]
without_fw = feat_matrix[:,[1,2,3,4,5]]
without_profanity = feat_matrix[:,[0,2,3,4,5]]
without_avg_sent = feat_matrix[:,[0,1,3,4,5]]
without_yules = feat_matrix[:,[0,1,2,3,4]]

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
X_train, X_test, y_train, y_test = train_test_split(feat_matrix, labels, test_size=0.2, random_state=42)

SVM

In [60]:
from sklearn import svm
from sklearn.metrics import accuracy_score

In [61]:
svm_clf = svm.SVC()

In [62]:
svm_clf.fit(X_train, y_train)

SVC()

In [63]:
prediction = svm_clf.predict(X_test)

In [64]:
accuracy_score(y_test, prediction)

0.6170886075949367

In [65]:
svm_clf.dual_coef_

array([[-1., -1., -1., ...,  1.,  1.,  1.]])

Random Forest

In [66]:
from sklearn.ensemble import RandomForestClassifier

In [67]:
random_forest_clf = RandomForestClassifier(random_state=42)

In [68]:
random_forest_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [69]:
prediction = random_forest_clf.predict(X_test)

In [70]:
random_forest_clf.feature_importances_

array([0.32397729, 0.04579973, 0.16396865, 0.15928598, 0.12239722,
       0.11438042, 0.0701907 ])

In [71]:
accuracy_score(y_test, prediction) #random_state decreased the accuracy

0.6962025316455697

# Notes on feature combinations: 

When train/test split is 80/20

Features = function words, profanity words, avg sentence length, avg word length, lix, yules i, number of misspellings difference


**All:** 
    SVM acc = **0.617**0886075949367,
    RF acc = **0.724**6835443037974 

**Without number of misspellings difference:** 
    SVM acc = **0.648**7341772151899,
    RF acc = **0.721**5189873417721
    
**Without lix:** 
    SVM acc = **0.655**0632911392406,
    RF acc = **0.727**8481012658228
    
**Without yules I:** 
    SVM acc = **0.645**5696202531646,
    RF acc = **0.712**0253164556962

**Without average sentence length:**
    SVM acc = **0.563**2911392405063,
    RF acc = **0.718**3544303797469

**Without average word length:**
    SVM acc = **0.642**4050632911392,
    RF acc = **0.674**0506329113924
    
**Without function words:** 
    SVM acc = **0.645**5696202531646,
    RF acc = **0.648**7341772151899
    
**Without profanity words:** 
    SVM acc = **0.645**5696202531646,
    RF acc = **0.699**3670886075949 

In [73]:
corpus[1426]

"i֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒֒